In [ ]:
import pip
pip.main(["install", "openpyxl"])

In [1]:
import pandas as pd
# import openpyxl

In [50]:
df = pd.read_excel("../../data/production/WMD2022 Commodities 1984-2020 extraInfo_Litio_USGS.xlsx")
df

,Minerals in metric tons*,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2015,2016,2017,2018,2019,2020,Es inicio,Kilo,cmdCode,Version
0,Iron Ore (Fe-cont.),446887838.0,492697409.0,495584899.0,507784666.0,524316505.0,539637227.0,529023870.0,531831265.0,499087427.0,...,1.456311e+09,1.466177e+09,1.511759e+09,1.523930e+09,1.552366e+09,1.522559e+09,1.0,NaN,2601.0,org
1,Albania,439502.0,414035.0,379154.0,444690.0,488050.0,539314.0,425697.0,203830.0,10000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,org
2,Algeria,2000000.0,1836000.0,1836000.0,1836000.0,1674000.0,1483920.0,1588140.0,1266000.0,1362400.0,...,5.098000e+05,3.283000e+05,2.684000e+05,3.812000e+05,4.099000e+05,4.066000e+05,NaN,NaN,NaN,org
3,Argentina,345576.0,388567.0,514259.0,359902.0,378576.0,414425.0,444115.0,88932.0,2502.0,...,2.060600e+05,1.072800e+05,2.840000e+04,4.056000e+04,0.000000e+00,0.000000e+00,NaN,NaN,NaN,org
4,Australia,56885000.0,62042117.0,60081983.0,64798343.0,61058000.0,66660300.0,66402630.0,74133990.0,72075000.0,...,5.009936e+08,5.310754e+08,5.482971e+08,5.600546e+08,5.679571e+08,5.683846e+08,NaN,NaN,NaN,org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2674,* Minerals in metric tons except,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2675,"Rhenium, Precious Metals in kg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Minerals in metric tons except
### Rhenium, Precious Metals in kg
### Natural Gas in Million m3


In [ ]:
# Get headers
header_indexes = df.index[df["Es inicio"] == 1.0].tolist()

def convert_df(ugly_df):
    # We get
    #   - name of the material
    #   - commodity code
    #   - if we have to handle it as kilogram or 1,000 kg
    #   - if it's the second version of lithium (USGS)
    # from first column and first row
    mineral_name, cmdCode, isKg, source = ugly_df.iloc[0, 0], ugly_df.iloc[0]["cmdCode"], ugly_df.iloc[0]["Kilo"], ugly_df.iloc[0]["Version"]

    # Get rid of "Es inicio" column
    del ugly_df["Es inicio"]
    del ugly_df["Kilo"]
    del ugly_df["cmdCode"]
    del ugly_df["Version"]

    # Remove the 1st cell since it's a header
    tmp_df = ugly_df.iloc[1:]
    # Add column with the name of mineral/thing
    tmp_df["Product"] = mineral_name
    tmp_df["cmdCode"] = cmdCode
    tmp_df["Source"] = source

    # Change the 1st column as Country
    tmp_df.rename(columns={"Minerals in metric tons*": "Country"}, inplace=True)
    # Melt the df into a conventional table    
    melted_df = tmp_df.melt(
        id_vars=["Country", "Product", "cmdCode", "Source"],
        var_name="Year",
        value_name="Value"
    )
    # If isKg flag not equals to 1 that means we have to multiply it by 1,000 because of is considered as a ton
    if isKg != 1:
        melted_df["Value (Ton)"] = melted_df["Value"]
        melted_df["Value"] = melted_df["Value"] * 1000
    else:
        melted_df["Value (Ton)"] = melted_df["Value"] / 1000
    return melted_df

list_df = list()
for n in range(len(header_indexes) - 1):
    # print(header_indexes[n], header_indexes[n + 1])
    tmp_df = df.iloc[header_indexes[n]:header_indexes[n + 1]]
    melted_df = convert_df(tmp_df)
    list_df.append(melted_df)
# and the last one. Skip last 5 rows because of the annotations
tmp_df = df.iloc[header_indexes[-1]:df.shape[0] - 5]
melted_df = convert_df(tmp_df)
list_df.append(melted_df)


In [ ]:
list_df[0]

In [ ]:
list_df[27]

In [73]:
list_df[-1]

,Country,Product,cmdCode,Source,Year,Value,Value (Ton)
0,Argentina,Uranium (U3O8-cont.),261210.0,org,1984,151000.0,151.0
1,Australia,Uranium (U3O8-cont.),261210.0,org,1984,5099000.0,5099.0
2,Belgium,Uranium (U3O8-cont.),261210.0,org,1984,59000.0,59.0
3,Brazil,Uranium (U3O8-cont.),261210.0,org,1984,138000.0,138.0
4,Bulgaria,Uranium (U3O8-cont.),261210.0,org,1984,0.0,0.0
...,...,...,...,...,...,...,...
1179,Spain,Uranium (U3O8-cont.),261210.0,org,2020,NaN,NaN
1180,Ukraine,Uranium (U3O8-cont.),261210.0,org,2020,877000.0,877.0
1181,United States,Uranium (U3O8-cont.),261210.0,org,2020,7000.0,7.0
1182,Uzbekistan,Uranium (U3O8-cont.),261210.0,org,2020,4127000.0,4127.0


In [71]:
len(list_df)

67

## Convert the list of dataframes into only one DF

In [88]:
final_production_df = pd.concat(list_df, ignore_index=True)
final_production_df["qtyUnitAbbr"] = "kg"

In [89]:
final_production_df

,Country,Product,cmdCode,Source,Year,Value,Value (Ton),qtyUnitAbbr
0,Albania,Iron Ore (Fe-cont.),2601.0,org,1984,4.395020e+08,439502.0,kg
1,Algeria,Iron Ore (Fe-cont.),2601.0,org,1984,2.000000e+09,2000000.0,kg
2,Argentina,Iron Ore (Fe-cont.),2601.0,org,1984,3.455760e+08,345576.0,kg
3,Australia,Iron Ore (Fe-cont.),2601.0,org,1984,5.688500e+10,56885000.0,kg
4,Austria,Iron Ore (Fe-cont.),2601.0,org,1984,1.137872e+09,1137872.0,kg
...,...,...,...,...,...,...,...,...
96380,Spain,Uranium (U3O8-cont.),261210.0,org,2020,NaN,NaN,kg
96381,Ukraine,Uranium (U3O8-cont.),261210.0,org,2020,8.770000e+05,877.0,kg
96382,United States,Uranium (U3O8-cont.),261210.0,org,2020,7.000000e+03,7.0,kg
96383,Uzbekistan,Uranium (U3O8-cont.),261210.0,org,2020,4.127000e+06,4127.0,kg


## Asign reporter code to the country

In [90]:
# del final_production_df["reporterCode"]
# del final_production_df["reporterDesc"]

In [91]:
final_production_df["Country"] = final_production_df["Country"].str.upper()
final_production_df["Product"] = final_production_df["Product"].str.upper()

# Attach country code to df
countries_list = [{'reporterCode': 36, 'reporterDesc': 'Australia'},
 {'reporterCode': 40, 'reporterDesc': 'Austria'},
 {'reporterCode': 56, 'reporterDesc': 'Belgium'},
 {'reporterCode': 68, 'reporterDesc': 'Bolivia (Plurinational State of)'},
 {'reporterCode': 68, 'reporterDesc': 'BOLIVIA, PLURINATIONAL STATE OF'},
 {'reporterCode': 68, 'reporterDesc': 'Bolivia'},
 {'reporterCode': 76, 'reporterDesc': 'Brazil'},
 {'reporterCode': 156, 'reporterDesc': 'China'},
 {'reporterCode': 251, 'reporterDesc': 'France'},
 {'reporterCode': 276, 'reporterDesc': 'Germany'},
 {'reporterCode': 278, 'reporterDesc': 'GERMAN DEM. REP'},
 {'reporterCode': 320, 'reporterDesc': 'Guatemala'},
 {'reporterCode': 360, 'reporterDesc': 'Indonesia'},
 {'reporterCode': 380, 'reporterDesc': 'Italy'},
 {'reporterCode': 417, 'reporterDesc': 'Kyrgyzstan'},
 {'reporterCode': 504, 'reporterDesc': 'Morocco'},
 {'reporterCode': 528, 'reporterDesc': 'Netherlands'},
 {'reporterCode': 710, 'reporterDesc': 'South Africa'},
 {'reporterCode': 757, 'reporterDesc': 'Switzerland'},
 {'reporterCode': 764, 'reporterDesc': 'Thailand'},
 {'reporterCode': 792, 'reporterDesc': 'Türkiye'},
 {'reporterCode': 807, 'reporterDesc': 'North Macedonia'},
 {'reporterCode': 807, 'reporterDesc': 'MACEDONIA'},
 {'reporterCode': 842, 'reporterDesc': 'USA'},
 {'reporterCode': 842, 'reporterDesc': 'UNITED STATES'},
 
 {'reporterCode': 32, 'reporterDesc': 'Argentina'},
 {'reporterCode': 100, 'reporterDesc': 'Bulgaria'},
 {'reporterCode': 112, 'reporterDesc': 'Belarus'},
 {'reporterCode': 124, 'reporterDesc': 'Canada'},
 {'reporterCode': 144, 'reporterDesc': 'Sri Lanka'},
 {'reporterCode': 152, 'reporterDesc': 'Chile'},
 {'reporterCode': 170, 'reporterDesc': 'Colombia'},
 {'reporterCode': 196, 'reporterDesc': 'Cyprus'},
 {'reporterCode': 222, 'reporterDesc': 'El Salvador'},
 {'reporterCode': 233, 'reporterDesc': 'Estonia'},
 {'reporterCode': 300, 'reporterDesc': 'Greece'},
 {'reporterCode': 372, 'reporterDesc': 'Ireland'},
 {'reporterCode': 398, 'reporterDesc': 'Kazakhstan'},
 {'reporterCode': 404, 'reporterDesc': 'Kenya'},
 {'reporterCode': 410, 'reporterDesc': 'Rep. of Korea'},
 {'reporterCode': 410, 'reporterDesc': 'KOREA, REPUBLIC OF'},
 {'reporterCode': 410, 'reporterDesc': 'KOREA, SOUTH'},
 {'reporterCode': 440, 'reporterDesc': 'Lithuania'},
 {'reporterCode': 484, 'reporterDesc': 'Mexico'},
 {'reporterCode': 512, 'reporterDesc': 'Oman'},
 {'reporterCode': 516, 'reporterDesc': 'Namibia'},
 {'reporterCode': 604, 'reporterDesc': 'Peru'},
 {'reporterCode': 620, 'reporterDesc': 'Portugal'},
 {'reporterCode': 642, 'reporterDesc': 'Romania'},
 {'reporterCode': 643, 'reporterDesc': 'Russian Federation'},
 {'reporterCode': 643, 'reporterDesc': 'RUSSIA'}, 
 {'reporterCode': 682, 'reporterDesc': 'Saudi Arabia'},
 {'reporterCode': 686, 'reporterDesc': 'Senegal'},
 {'reporterCode': 699, 'reporterDesc': 'India'},
 {'reporterCode': 702, 'reporterDesc': 'Singapore'},
 {'reporterCode': 716, 'reporterDesc': 'Zimbabwe'},
 {'reporterCode': 724, 'reporterDesc': 'Spain'},
 {'reporterCode': 748, 'reporterDesc': 'Eswatini'},
 {'reporterCode': 752, 'reporterDesc': 'Sweden'},
 {'reporterCode': 780, 'reporterDesc': 'Trinidad and Tobago'},
 {'reporterCode': 784, 'reporterDesc': 'United Arab Emirates'},
 {'reporterCode': 788, 'reporterDesc': 'Tunisia'},
 {'reporterCode': 795, 'reporterDesc': 'Turkmenistan'},
 {'reporterCode': 804, 'reporterDesc': 'Ukraine'},
 {'reporterCode': 826, 'reporterDesc': 'United Kingdom'},
 {'reporterCode': 891, 'reporterDesc': 'Serbia and Montenegro (...2005)'},
 {'reporterCode': 31, 'reporterDesc': 'Azerbaijan'},
 {'reporterCode': 44, 'reporterDesc': 'Bahamas'},
 {'reporterCode': 120, 'reporterDesc': 'Cameroon'},
 {'reporterCode': 191, 'reporterDesc': 'Croatia'},
 {'reporterCode': 203, 'reporterDesc': 'Czechia'},
 {'reporterCode': 203, 'reporterDesc': 'CZECH REPUBLIC'},
 {'reporterCode': 208, 'reporterDesc': 'Denmark'},
 {'reporterCode': 246, 'reporterDesc': 'Finland'},
 {'reporterCode': 266, 'reporterDesc': 'Gabon'},
 {'reporterCode': 268, 'reporterDesc': 'Georgia'},
 {'reporterCode': 344, 'reporterDesc': 'China, Hong Kong SAR'},
 {'reporterCode': 344, 'reporterDesc': 'HONG KONG'},
 {'reporterCode': 364, 'reporterDesc': 'Iran'},
 {'reporterCode': 364, 'reporterDesc': 'IRAN, ISLAMIC REPUBLIC OF'},
 {'reporterCode': 392, 'reporterDesc': 'Japan'},
 {'reporterCode': 422, 'reporterDesc': 'Lebanon'},
 {'reporterCode': 480, 'reporterDesc': 'Mauritius'},
 {'reporterCode': 490, 'reporterDesc': 'Other Asia, nes'},
 {'reporterCode': 579, 'reporterDesc': 'Norway'},
 {'reporterCode': 703, 'reporterDesc': 'Slovakia'},
 {'reporterCode': 704, 'reporterDesc': 'Viet Nam'},
 {'reporterCode': 704, 'reporterDesc': 'VIETNAM'},
 {'reporterCode': 705, 'reporterDesc': 'Slovenia'},
 {'reporterCode': 862, 'reporterDesc': 'Venezuela'},
 {'reporterCode': 862, 'reporterDesc': 'VENEZUELA, BOLIVARIAN REPUBLIC O'},
 {'reporterCode': 192, 'reporterDesc': 'Cuba'},
 {'reporterCode': 450, 'reporterDesc': 'Madagascar'},
 {'reporterCode': 608, 'reporterDesc': 'Philippines'},
 {'reporterCode': 736, 'reporterDesc': 'Sudan (...2011)'},
 {'reporterCode': 8, 'reporterDesc': 'Albania'},
 {'reporterCode': 72, 'reporterDesc': 'Botswana'},
 {'reporterCode': 348, 'reporterDesc': 'Hungary'},
 {'reporterCode': 384, 'reporterDesc': "Côte d'Ivoire"},
 {'reporterCode': 384, 'reporterDesc': "COTE D'IVOIRE"},
 {'reporterCode': 400, 'reporterDesc': 'Jordan'},
 {'reporterCode': 428, 'reporterDesc': 'Latvia'},
 {'reporterCode': 442, 'reporterDesc': 'Luxembourg'},
 {'reporterCode': 458, 'reporterDesc': 'Malaysia'},
 {'reporterCode': 470, 'reporterDesc': 'Malta'},
 {'reporterCode': 496, 'reporterDesc': 'Mongolia'},
 {'reporterCode': 508, 'reporterDesc': 'Mozambique'},
 {'reporterCode': 554, 'reporterDesc': 'New Zealand'},
 {'reporterCode': 562, 'reporterDesc': 'Niger'},
 {'reporterCode': 616, 'reporterDesc': 'Poland'},
 {'reporterCode': 634, 'reporterDesc': 'Qatar'},
 {'reporterCode': 659, 'reporterDesc': 'Saint Kitts and Nevis'},
 {'reporterCode': 740, 'reporterDesc': 'Suriname'},
 {'reporterCode': 818, 'reporterDesc': 'Egypt'},
 {'reporterCode': 894, 'reporterDesc': 'Zambia'},
 {'reporterCode': 51, 'reporterDesc': 'Armenia'},
 {'reporterCode': 340, 'reporterDesc': 'Honduras'},
 {'reporterCode': 466, 'reporterDesc': 'Mali'},
 {'reporterCode': 598, 'reporterDesc': 'Papua New Guinea'},
 {'reporterCode': 20, 'reporterDesc': 'Andorra'},
 {'reporterCode': 108, 'reporterDesc': 'Burundi'},
 {'reporterCode': 116, 'reporterDesc': 'Cambodia'},
 {'reporterCode': 140, 'reporterDesc': 'Central African Rep.'},
 {'reporterCode': 140, 'reporterDesc': 'CENTRAL AFRICAN REPUBLIC'},
 {'reporterCode': 188, 'reporterDesc': 'Costa Rica'},
 {'reporterCode': 204, 'reporterDesc': 'Benin'},
 {'reporterCode': 218, 'reporterDesc': 'Ecuador'},
 {'reporterCode': 231, 'reporterDesc': 'Ethiopia'},
 {'reporterCode': 242, 'reporterDesc': 'Fiji'},
 {'reporterCode': 258, 'reporterDesc': 'French Polynesia'},
 {'reporterCode': 270, 'reporterDesc': 'Gambia'},
 {'reporterCode': 288, 'reporterDesc': 'Ghana'},
 {'reporterCode': 324, 'reporterDesc': 'Guinea'},
 {'reporterCode': 328, 'reporterDesc': 'Guyana'},
 {'reporterCode': 376, 'reporterDesc': 'Israel'},
 {'reporterCode': 414, 'reporterDesc': 'Kuwait'},
 {'reporterCode': 558, 'reporterDesc': 'Nicaragua'},
 {'reporterCode': 591, 'reporterDesc': 'Panama'},
 {'reporterCode': 762, 'reporterDesc': 'Tajikistan'},
 {'reporterCode': 834, 'reporterDesc': 'United Rep. of Tanzania'},
 {'reporterCode': 834, 'reporterDesc': 'TANZANIA, UNITED REPUBLIC OF'},
 {'reporterCode': 834, 'reporterDesc': 'TANZANIA'}, 
 {'reporterCode': 854, 'reporterDesc': 'Burkina Faso'},
 {'reporterCode': 858, 'reporterDesc': 'Uruguay'},
 {'reporterCode': 132, 'reporterDesc': 'Cabo Verde'},
 {'reporterCode': 132, 'reporterDesc': 'CAPE VERDE'},
 {'reporterCode': 48, 'reporterDesc': 'Bahrain'},
 {'reporterCode': 388, 'reporterDesc': 'Jamaica'},
 {'reporterCode': 478, 'reporterDesc': 'Mauritania'},
 {'reporterCode': 12, 'reporterDesc': 'Algeria'},
 {'reporterCode': 28, 'reporterDesc': 'Antigua and Barbuda'},
 {'reporterCode': 540, 'reporterDesc': 'New Caledonia'},
 {'reporterCode': 52, 'reporterDesc': 'Barbados'},
 {'reporterCode': 566, 'reporterDesc': 'Nigeria'},
 {'reporterCode': 800, 'reporterDesc': 'Uganda'},
 {'reporterCode': 426, 'reporterDesc': 'Lesotho'},
 {'reporterCode': 174, 'reporterDesc': 'Comoros'},
 {'reporterCode': 50, 'reporterDesc': 'Bangladesh'},
 {'reporterCode': 352, 'reporterDesc': 'Iceland'},
 {'reporterCode': 454, 'reporterDesc': 'Malawi'},


 {'reporterCode': 662, 'reporterDesc': 'Saint Lucia'},
 {'reporterCode': 214, 'reporterDesc': 'Dominican Rep.'},
 {'reporterCode': 214, 'reporterDesc': 'DOMINICAN REPUBLIC'},
 {'reporterCode': 882, 'reporterDesc': 'Samoa'},
 {'reporterCode': 646, 'reporterDesc': 'Rwanda'},
 {'reporterCode': 96, 'reporterDesc': 'Brunei Darussalam'},
 {'reporterCode': 96, 'reporterDesc': 'BRUNEI'},
 {'reporterCode': 776, 'reporterDesc': 'Tonga'},
 {'reporterCode': 600, 'reporterDesc': 'Paraguay'},
 {'reporterCode': 760, 'reporterDesc': 'Syria'},
 {'reporterCode': 760, 'reporterDesc': 'SYRIAN ARAB REPUBLIC'},
 {'reporterCode': 498, 'reporterDesc': 'Rep. of Moldova'},
 {'reporterCode': 498, 'reporterDesc': 'MOLDOVA, REPUBLIC OF'},
 {'reporterCode': 498, 'reporterDesc': 'MOLDOVA'},
 {'reporterCode': 768, 'reporterDesc': 'Togo'},
 {'reporterCode': 90, 'reporterDesc': 'Solomon Isds'},
 {'reporterCode': 90, 'reporterDesc': 'SOLOMON ISLANDS'},
 {'reporterCode': 690, 'reporterDesc': 'Seychelles'},
 {'reporterCode': 70, 'reporterDesc': 'Bosnia Herzegovina'},
 {'reporterCode': 70, 'reporterDesc': 'BOSNIA-HERZEGOVINA'},
 {'reporterCode': 70, 'reporterDesc': 'BOSNIA AND HERZEGOVINA'},
 {'reporterCode': 500, 'reporterDesc': 'Montserrat'},
 {'reporterCode': 586, 'reporterDesc': 'Pakistan'},
 {'reporterCode': 524, 'reporterDesc': 'Nepal'},
 {'reporterCode': 887, 'reporterDesc': 'Yemen'},
 {'reporterCode': 887, 'reporterDesc': 'YEMEN ARAB REPUBLIC'}, # norte
 {'reporterCode': 887, 'reporterDesc': 'YEMEN, PDR'},
 {'reporterCode': 626, 'reporterDesc': 'Timor-Leste'},
 {'reporterCode': 798, 'reporterDesc': 'Tuvalu'},
 {'reporterCode': 64, 'reporterDesc': 'Bhutan'},
 {'reporterCode': 670, 'reporterDesc': 'Saint Vincent and the Grenadines'},
 {'reporterCode': 184, 'reporterDesc': 'Cook Isds'},
 {'reporterCode': 84, 'reporterDesc': 'Belize'},
 {'reporterCode': 530, 'reporterDesc': 'Netherlands Antilles (...2010)'},
 {'reporterCode': 688, 'reporterDesc': 'Serbia'},
 {'reporterCode': 548, 'reporterDesc': 'Vanuatu'},
 {'reporterCode': 275, 'reporterDesc': 'State of Palestine'},
 {'reporterCode': 434, 'reporterDesc': 'Libya'},
 {'reporterCode': 434, 'reporterDesc': 'LIBYAN ARAB JAMAHIRIYA'},
 {'reporterCode': 178, 'reporterDesc': 'Congo'},
 {'reporterCode': 178, 'reporterDesc': 'CONGO, REP.'},
 {'reporterCode': 24, 'reporterDesc': 'Angola'},
 {'reporterCode': 296, 'reporterDesc': 'Kiribati'},
 {'reporterCode': 499, 'reporterDesc': 'Montenegro'},
 {'reporterCode': 234, 'reporterDesc': 'Faeroe Isds'},
 {'reporterCode': 533, 'reporterDesc': 'Aruba'},
 {'reporterCode': 796, 'reporterDesc': 'Turks and Caicos Isds'},
 {'reporterCode': 104, 'reporterDesc': 'Myanmar'},
 {'reporterCode': 212, 'reporterDesc': 'Dominica'},
 {'reporterCode': 418, 'reporterDesc': "Lao People's Dem. Rep."},
 {'reporterCode': 418, 'reporterDesc': "LAO PEOPLE'S DEMOCRATIC REPUBLIC"},
 {'reporterCode': 418, 'reporterDesc': "LAOS"},

 {'reporterCode': 304, 'reporterDesc': 'Greenland'},
 {'reporterCode': 368, 'reporterDesc': 'Iraq'},
 
 {'reporterCode': 729, 'reporterDesc': 'Sudan'},
 {'reporterCode': 446, 'reporterDesc': 'China, Macao SAR'},
 {'reporterCode': 308, 'reporterDesc': 'Grenada'},
 {'reporterCode': 585, 'reporterDesc': 'Palau'},
 {'reporterCode': 694, 'reporterDesc': 'Sierra Leone'},
 {'reporterCode': 60, 'reporterDesc': 'Bermuda'},
 {'reporterCode': 4, 'reporterDesc': 'Afghanistan'},
 {'reporterCode': 180, 'reporterDesc': 'Dem. Rep. of the Congo'},
 {'reporterCode': 180, 'reporterDesc': 'CONGO, D.R.'},
 {'reporterCode': 180, 'reporterDesc': 'CONGO, THE DEMOCRATIC REPUBLIC O'},
 {'reporterCode': 860, 'reporterDesc': 'Uzbekistan'},
 {'reporterCode': 136, 'reporterDesc': 'Cayman Isds'},

 {'reporterCode': 430, 'reporterDesc': 'Liberia'},
 {'reporterCode': 891, 'reporterDesc': 'Serbia and montenegro'},
 {'reporterCode': 332, 'reporterDesc': 'HAITI'},
 {'reporterCode': 792, 'reporterDesc': 'TURKEY'},
 {'reporterCode': 890, 'reporterDesc': 'FMR YUGOSLAVIA'},
 {'reporterCode': 890, 'reporterDesc': 'YUGOSLAVIA'},
 {'reporterCode': 200, 'reporterDesc': 'CZECHOSLOVAKIA'},
#  {'reporterCode': , 'reporterDesc': 'FMR RHODESIA NYAS'},
 {'reporterCode': 748, 'reporterDesc': 'SWAZILAND'},
 {'reporterCode': 408, 'reporterDesc': "DEM. PEOPLE'S REP. OF KOREA"},
 {'reporterCode': 408, 'reporterDesc': "KOREA, NORTH"},
 {'reporterCode': 408, 'reporterDesc': "KOREA, DEMOCRATIC PEOPLE'S REPUB"},
 {'reporterCode': 807, 'reporterDesc': "TFYR OF MACEDONIA"},
#  {'reporterCode': , 'reporterDesc': "OTHER AFRICA, NES"},
#  {'reporterCode': , 'reporterDesc': 'OTHER EUROPE, NES'},
 {'reporterCode': 232, 'reporterDesc': "ERITREA"},
 {'reporterCode': 148, 'reporterDesc': "CHAD"},
 {'reporterCode': 226, 'reporterDesc': "EQUATORIAL GUINEA"},
 {'reporterCode': 254, 'reporterDesc': "FRENCH GUIANA"},
 {'reporterCode': 736, 'reporterDesc': "SOUTH SUDAN"},
 {'reporterCode': 706, 'reporterDesc': "SOMALIA"},
#  {'reporterCode': , 'reporterDesc': ""},
 ]
countries_list = [{"reporterCode": country["reporterCode"], "Country": country["reporterDesc"].upper()} for country in countries_list]
df_countries_code = pd.json_normalize(countries_list)

final_production_df = pd.merge(final_production_df, df_countries_code, how="left", left_on="Country", right_on="Country")
final_production_df

,Country,Product,cmdCode,Source,Year,Value,Value (Ton),qtyUnitAbbr,reporterCode
0,ALBANIA,IRON ORE (FE-CONT.),2601.0,org,1984,4.395020e+08,439502.0,kg,8.0
1,ALGERIA,IRON ORE (FE-CONT.),2601.0,org,1984,2.000000e+09,2000000.0,kg,12.0
2,ARGENTINA,IRON ORE (FE-CONT.),2601.0,org,1984,3.455760e+08,345576.0,kg,32.0
3,AUSTRALIA,IRON ORE (FE-CONT.),2601.0,org,1984,5.688500e+10,56885000.0,kg,36.0
4,AUSTRIA,IRON ORE (FE-CONT.),2601.0,org,1984,1.137872e+09,1137872.0,kg,40.0
...,...,...,...,...,...,...,...,...,...
96380,SPAIN,URANIUM (U3O8-CONT.),261210.0,org,2020,NaN,NaN,kg,724.0
96381,UKRAINE,URANIUM (U3O8-CONT.),261210.0,org,2020,8.770000e+05,877.0,kg,804.0
96382,UNITED STATES,URANIUM (U3O8-CONT.),261210.0,org,2020,7.000000e+03,7.0,kg,842.0
96383,UZBEKISTAN,URANIUM (U3O8-CONT.),261210.0,org,2020,4.127000e+06,4127.0,kg,860.0


In [92]:
# DATA with no reporter code
final_production_df[final_production_df["reporterCode"].isna()]["Country"].unique()

array(['KOSOVO', 'ZAIRE', 'TAIWAN', 'CHRISTMAS ISLAND', 'NAURU',
       'PUERTO RICO'], dtype=object)

## Save it as csv

In [93]:
final_production_df.to_csv("../../data/production/Production_one_file_2.csv", index=False)

In [94]:
len(final_production_df["Country"].unique()) * 37 * 67

443741